#### Imported necessary libraries

In [1]:
import re
from email.parser import HeaderParser

import joblib
import nltk
import pandas as pd
from bs4 import BeautifulSoup

from DTO.EmailData import EmailData
from Parsers.EmailParser import emailParser

#### Pandas ENV Variables

In [17]:
pd.set_option("display.max_colwidth", 200)

#### Transform the email headers from string to pandas DataFrame

In [13]:
e = 'Received: from AS8P189MB1287.EURP189.PROD.OUTLOOK.COM (2603:10a6:20b:2a6::12)\r\n by DB9P189MB1833.EURP189.PROD.OUTLOOK.COM with HTTPS; Fri, 26 Jan 2024\r\n 11:55:17 +0000\r\nReceived: from AM0PR06CA0095.eurprd06.prod.outlook.com (2603:10a6:208:fa::36)\r\n by AS8P189MB1287.EURP189.PROD.OUTLOOK.COM (2603:10a6:20b:2a6::12) with\r\n Microsoft SMTP Server (version=TLS1_2,\r\n cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.7228.22; Fri, 26 Jan\r\n 2024 11:55:16 +0000\r\nReceived: from VI1EUR05FT068.eop-eur05.prod.protection.outlook.com\r\n (2603:10a6:208:fa:cafe::32) by AM0PR06CA0095.outlook.office365.com\r\n (2603:10a6:208:fa::36) with Microsoft SMTP Server (version=TLS1_2,\r\n cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id 15.20.7228.27 via Frontend\r\n Transport; Fri, 26 Jan 2024 11:55:16 +0000\r\nAuthentication-Results: spf=pass (sender IP is 209.85.128.169)\r\n smtp.mailfrom=gmail.com; dkim=pass (signature was verified)\r\n header.d=gmail.com;dmarc=pass action=none header.from=gmail.com;compauth=pass\r\n reason=100\r\nReceived-SPF: Pass (protection.outlook.com: domain of gmail.com designates\r\n 209.85.128.169 as permitted sender) receiver=protection.outlook.com;\r\n client-ip=209.85.128.169; helo=mail-yw1-f169.google.com; pr=C\r\nReceived: from mail-yw1-f169.google.com (209.85.128.169) by\r\n VI1EUR05FT068.mail.protection.outlook.com (10.233.242.179) with Microsoft\r\n SMTP Server (version=TLS1_2, cipher=TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384) id\r\n 15.20.7228.21 via Frontend Transport; Fri, 26 Jan 2024 11:55:16 +0000\r\nX-IncomingTopHeaderMarker:\r\n OriginalChecksum:193C4CC6070001D490168F7408E4373170375527C24E979596F1CD7AB0DA98C4;UpperCasedChecksum:0738A38079E727F4133772F8E54A3D98E23260AA28691F03BEC46B0AEE93E353;SizeAsReceived:2404;Count:13\r\nReceived: by mail-yw1-f169.google.com with SMTP id 00721157ae682-602cf45bfe9so2783637b3.1\r\n        for <testaddin2024@outlook.com>; Fri, 26 Jan 2024 03:55:16 -0800 (PST)\r\nDKIM-Signature: v=1; a=rsa-sha256; c=relaxed/relaxed;\r\n        d=gmail.com; s=20230601; t=1706270116; x=1706874916; darn=outlook.com;\r\n        h=to:subject:message-id:date:from:mime-version:from:to:cc:subject\r\n         :date:message-id:reply-to;\r\n        bh=8tBB2/x2+X6ru2G9+3dhXEw8edOgrRA8xcfehfwxlj4=;\r\n        b=NL+VldBbQEk7gVv3+HDrhTyJX6CUaHE704A1zQPvbypmeMEHV7TUtnvv1PUL6ZP5Ec\r\n         sz8AaL/fJvxgpW9v+iAsZis1qGbevfrvMc+fZTyAMh7WG/fjEkuZqApcZa9N0lg6Hn28\r\n         UASP79s42yMotThbbCCkDas6XoxZ3+ZpSQxVGXWfDAzE1BwsJcslewFNSlNZSkGKZ7Xi\r\n         kMedcSUmEbGHcHZ1DxnZ/+4nOj911ixXPYtOguTg4oKimdEQ/MjX6QPH2v/5kGKMdjFU\r\n         YW/Krhg5l6+oyC3odyl9H+odJOWGvCuTAct/JiH8w+H6Wh5PlRguUA7Sl3wB9922OtwI\r\n         JN9w==\r\nX-Google-DKIM-Signature: v=1; a=rsa-sha256; c=relaxed/relaxed;\r\n        d=1e100.net; s=20230601; t=1706270116; x=1706874916;\r\n        h=to:subject:message-id:date:from:mime-version:x-gm-message-state\r\n         :from:to:cc:subject:date:message-id:reply-to;\r\n        bh=8tBB2/x2+X6ru2G9+3dhXEw8edOgrRA8xcfehfwxlj4=;\r\n        b=CAtGcQsvWLs3cTQmmimA2LZDJLsqcXuAARwMOLzQyAbbA/cpdTUBBqtOgt5L2diZkw\r\n         v26ymuFM5bCuxedasZ3q5cbFmQ0qVm7cf6TaH7omiTiGz2Wg9VSAYyJ0F4nqgD3Xu5q1\r\n         6GceDJavWmp4pmTZj9nEAUo+gTOgJHcSMlaxTkr0LAMvl5y5AoShYFL21sWOOPVclWgd\r\n         g4kFVsyAauUE/xnKJ+MUauajY+1qFcnDs+F4+xLBFnOeloODoEz82p5aoHCzPz3jCBNG\r\n         yXz6bbr++uTpTSEP4DLbtV+fXBMNrTgUsT+PepKjTlmkuF1W1a5EdDSC432zgdfZ/2rJ\r\n         /iQA==\r\nX-Gm-Message-State: AOJu0YwpG+IgPRQP2AkIbZ7G2wJ6UqBS1YEsHWSrn6JT6MXAd9XY3mJx\r\n\tmudTFgyoIjXvn3LJHU3RI7nSisv1gxSsrPAcgJeZTrgpMlgDu/KLh5vYCyPxadRkuyAgE8mRXVe\r\n\tU2awsVLEYVEuTd0Y+4BpWixFEzYO5O/25\r\nX-Google-Smtp-Source: AGHT+IGn7zomVQRqFVjQGQS7Rd2byvTsXPC5cKYQ5QrGsv4urojzVWp4lbdD0ENH5uH9xZ6MczRPHdw1RCa6P/+TSZE=\r\nX-Received: by 2002:a81:b203:0:b0:5ff:8420:ce7f with SMTP id\r\n q3-20020a81b203000000b005ff8420ce7fmr1061455ywh.49.1706270115799; Fri, 26 Jan\r\n 2024 03:55:15 -0800 (PST)\r\nMIME-Version: 1.0\r\nFrom: Dan Barbu <dan.alexandru2003@gmail.com>\r\nDate: Fri, 26 Jan 2024 13:55:03 +0200\r\nMessage-ID: <CAPSJEV1bheVp_QzHzWza0JWtAExjG88AzamV_UvzD7-0oEwj1g@mail.gmail.com>\r\nSubject: test\r\nTo: testaddin2024@outlook.com\r\nContent-Type: multipart/alternative; boundary="00000000000024a8ad060fd7f899"\r\nX-IncomingHeaderCount: 13\r\nReturn-Path: dan.alexandru2003@gmail.com\r\nX-MS-Exchange-Organization-ExpirationStartTime: 26 Jan 2024 11:55:16.5067\r\n (UTC)\r\nX-MS-Exchange-Organization-ExpirationStartTimeReason: OriginalSubmit\r\nX-MS-Exchange-Organization-ExpirationInterval: 1:00:00:00.0000000\r\nX-MS-Exchange-Organization-ExpirationIntervalReason: OriginalSubmit\r\nX-MS-Exchange-Organization-Network-Message-Id:\r\n 3336fed5-fe25-4142-be56-08dc1e65a98f\r\nX-EOPAttributedMessage: 0\r\nX-EOPTenantAttributedMessage: 84df9e7f-e9f6-40af-b435-aaaaaaaaaaaa:0\r\nX-MS-Exchange-Organization-MessageDirectionality: Incoming\r\nX-MS-PublicTrafficType: Email\r\nX-MS-TrafficTypeDiagnostic:\r\n VI1EUR05FT068:EE_|AS8P189MB1287:EE_|DB9P189MB1833:EE_\r\nX-MS-Exchange-Organization-AuthSource:\r\n VI1EUR05FT068.eop-eur05.prod.protection.outlook.com\r\nX-MS-Exchange-Organization-AuthAs: Anonymous\r\nX-MS-UserLastLogonTime: 1/26/2024 11:53:08 AM\r\nX-MS-Office365-Filtering-Correlation-Id: 3336fed5-fe25-4142-be56-08dc1e65a98f\r\nX-MS-Exchange-EOPDirect: true\r\nX-Sender-IP: 209.85.128.169\r\nX-SID-PRA: DAN.ALEXANDRU2003@GMAIL.COM\r\nX-SID-Result: PASS\r\nX-MS-Exchange-Organization-PCL: 2\r\nX-MS-Exchange-Organization-SCL: 1\r\nX-Microsoft-Antispam: BCL:0;\r\nX-MS-Exchange-CrossTenant-OriginalArrivalTime: 26 Jan 2024 11:55:16.4599\r\n (UTC)\r\nX-MS-Exchange-CrossTenant-Network-Message-Id: 3336fed5-fe25-4142-be56-08dc1e65a98f\r\nX-MS-Exchange-CrossTenant-Id: 84df9e7f-e9f6-40af-b435-aaaaaaaaaaaa\r\nX-MS-Exchange-CrossTenant-AuthSource:\r\n VI1EUR05FT068.eop-eur05.prod.protection.outlook.com\r\nX-MS-Exchange-CrossTenant-AuthAs: Anonymous\r\nX-MS-Exchange-CrossTenant-FromEntityHeader: Internet\r\nX-MS-Exchange-CrossTenant-RMS-PersistedConsumerOrg:\r\n 00000000-0000-0000-0000-000000000000\r\nX-MS-Exchange-Transport-CrossTenantHeadersStamped: AS8P189MB1287\r\nX-MS-Exchange-Transport-EndToEndLatency: 00:00:01.1635865\r\nX-MS-Exchange-Processed-By-BccFoldering: 15.20.7228.022\r\nX-Message-Info:\r\n\tqZelhIiYnPnB6LAB1lQWVsnsVuBEHeNCGotbB8QwclKQB3g4zhFPgco8aZBAAIvciYhP35bblMXC9MhFW1wqsyNLjFATgP4JMqdQvMqcnIrBc6aXmJJR/rVOmtDtmCmUzEOhNIgXZBV917Hg9AbL0L0QWe2xYn2vX4qB7uofphZwaTZX91QRiwO4RwinO4hTjHv5zYkVRGf8OWx0GafyVA==\r\nX-Message-Delivery: Vj0xLjE7dXM9MDtsPTA7YT0wO0Q9MTtHRD0yO1NDTD0z\r\nX-Microsoft-Antispam-Mailbox-Delivery:\r\n\tucf:0;jmr:0;ex:0;auth:1;dest:I;ENG:(5062000305)(920221119095)(90000117)(920221120095)(90005022)(91005020)(90014020)(91030020)(91040095)(9050020)(9100341)(944500132)(4810010)(4910033)(9575002)(10195002)(9320005)(120001);\r\nX-Microsoft-Antispam-Message-Info:\r\n\t=?us-ascii?Q?QjfQNLFWghh08HjvXhph2mm1eIUYEBsT+ki4wV+GK9ASpS2MD/WEzo/qdDBG?=\r\n =?us-ascii?Q?ValmYQu7uKxTHH0KNtCSQ5GjQhGdZtj4H19tkgUDtUvbiqOvCnpjm4Zgm4xj?=\r\n =?us-ascii?Q?NvbcrUc4SrZYFkp9ex93K17/sp1FlHuPB4M3+Nlvq72apYZ7mt2oIX2S/47v?=\r\n =?us-ascii?Q?cwKJAbBRh/mRC4UkEczSSeZwbcIUpTWMGrN9PNnlbvW/t47coqymZjlFLoqi?=\r\n =?us-ascii?Q?O5BANWZ0M41MmkhvJ7xTvVWweDzNsgWTf38tgDJNyHK3R3uSfn8qD+67/Rki?=\r\n =?us-ascii?Q?teFD6M4SOp65DMqV3KAthdcEYVd6y9DVKKVtxTAEgArlAJeslq+YEpYGDPMK?=\r\n =?us-ascii?Q?yY473R5Thv3VOfdc4qBL2mmnD54333wO1/1iGbfnAL0Vs3CfTTMGw2P6qTnv?=\r\n =?us-ascii?Q?wjbh4vSzmRwFWExB4ZamlUUX1Y4r0/8kvkgAm73BfYpJsESiUCap/qgVSDbZ?=\r\n =?us-ascii?Q?eWbSvD2jEFCZ9xv2lwZErdJG3TQQYJkDCuWEzKKIoPl3A7kejqW0v/CrlWJ6?=\r\n =?us-ascii?Q?0FADlZzVaW/rKOleIIsY2PoygbqvIINtH60CafwmLmKbds0MuQ65LRbF/noQ?=\r\n =?us-ascii?Q?1yZB/PxZfd3SW7JQG9Tkzyvyi+pTxqTCV+K1MKFj8fkgqhONsctWJW7T3GCl?=\r\n =?us-ascii?Q?55WZHc0Gti4SmEc8hSdqN2qzku4K2DDfN+khmZbK6TRnd1iDoVvw8qE1PKoL?=\r\n =?us-ascii?Q?4ydWlwQJMVeDuP8Rel5b53DtDn4Td1T/Tr88fUVrc+qoqXdf7xvLzC2ICkkY?=\r\n =?us-ascii?Q?v5a5bT1RYqwCqwpAFXcul6S7ht3nIX+9W5qXAgqRicAH7IGt3w/WOXWf79QI?=\r\n =?us-ascii?Q?rgv6+oKl1Wo4hcpx6Q7qtlXOJr0RKvKQTQQjjOppWnmt799SfxKXMAL1GjvK?=\r\n =?us-ascii?Q?9GdvHObmOvZBs/wvv6tJhfTrqQooG2u4CQr6eOqEuDEg9BSOv9Qdh5FUfcTD?=\r\n =?us-ascii?Q?MeDzN3HnFkoyxBLOLtCTSbwIzEZRlZ+dio7zWNDcyFERGlBIo4sqIOlMbWHN?=\r\n =?us-ascii?Q?iSWnfnO6lLx5ZqjDaSLpr6eYQKFCdNHSjNfYs1lP2/VzxHNMSqY+7y5QA1H/?=\r\n =?us-ascii?Q?JktlxJuqOoWXYpxkuiVNxx4Hi5owUbfTm6ieToBTJxOqZ+co5bvg3ychMtlk?=\r\n =?us-ascii?Q?44d+449OxvbF/MQvOvwumqYoy4ZaUqjNAF4r0bUQE8QIul+AQjP49pjZT/Ni?=\r\n =?us-ascii?Q?OVexSmUSEcwvS9SIWBQ1a18pK/m9glXbssoCIp9b/HqBq/zPLdA2SuGVigme?=\r\n =?us-ascii?Q?GaLZ3wxwtEhdrBo9XYVbjjuLV5hyGXL3UxO2w2yM3WOG51JUlCAoDqyIhniT?=\r\n =?us-ascii?Q?mIJEeIMC6eEgOLpr2A3KiyXQ?=\r\n'

body = "<style type=\"text/css\">\r\n<!--\r\n-->\r\n</style><div class=\"apiBody\"><div><table dir=\"ltr\"><tbody><tr><td id=\"x_i1\" style=\"padding:0; font-family:'Segoe UI Semibold','Segoe UI Bold','Segoe UI','Helvetica Neue Medium',Arial,sans-serif; font-size:17px; color:#707070\">Microsoft account</td></tr><tr><td id=\"x_i2\" style=\"padding:0; font-family:'Segoe UI Light','Segoe UI','Helvetica Neue Medium',Arial,sans-serif; font-size:41px; color:#2672ec\">New app(s) have access to your data</td></tr><tr><td id=\"x_i3\" style=\"padding:0; padding-top:25px; font-family:'Segoe UI',Tahoma,Verdana,Arial,sans-serif; font-size:14px; color:#2a2a2a\">Graph Explorer connected to the Microsoft account <a dir=\"ltr\" id=\"x_iAccount\" class=\"x_link\" href=\"mailto:te**4@outlook.com\" style=\"color:#2672ec; text-decoration:none\">te**4@outlook.com</a>.</td></tr><tr><td id=\"x_i4\" style=\"padding:0; padding-top:25px; font-family:'Segoe UI',Tahoma,Verdana,Arial,sans-serif; font-size:14px; color:#2a2a2a\">If you didn't grant this access, please remove the app(s) from your account.</td></tr><tr><td style=\"padding:0; padding-top:25px; font-family:'Segoe UI',Tahoma,Verdana,Arial,sans-serif; font-size:14px; color:#2a2a2a\"><table border=\"0\" cellspacing=\"0\"><tbody><tr><td bgcolor=\"#2672ec\" style=\"background-color:#2672ec; padding-top:5px; padding-right:20px; padding-bottom:5px; padding-left:20px; min-width:50px\"><a id=\"x_i5\" href=\"https://account.live.com/consent/Manage?fn=email\" style=\"font-family:'Segoe UI Semibold','Segoe UI Bold','Segoe UI','Helvetica Neue Medium',Arial,sans-serif; font-size:14px; text-align:center; text-decoration:none; font-weight:600; letter-spacing:0.02em; color:#fff\">Manage your apps</a></td></tr></tbody></table></td></tr><tr><td id=\"x_i6\" style=\"padding:0; padding-top:25px; font-family:'Segoe UI',Tahoma,Verdana,Arial,sans-serif; font-size:14px; color:#2a2a2a\">You can also <a id=\"x_iLink3\" class=\"x_link\" href=\"https://account.live.com/SecurityNotifications/Update\" style=\"color:#2672ec; text-decoration:none\">opt out</a> or change where you receive security notifications.</td></tr><tr><td id=\"x_i7\" style=\"padding:0; padding-top:25px; font-family:'Segoe UI',Tahoma,Verdana,Arial,sans-serif; font-size:14px; color:#2a2a2a\">Thanks,</td></tr><tr><td id=\"x_i8\" style=\"padding:0; font-family:'Segoe UI',Tahoma,Verdana,Arial,sans-serif; font-size:14px; color:#2a2a2a\">The Microsoft account team</td></tr></tbody></table><div lang=\"en\" style=\"margin-top:20px; margin-bottom:10px\"><a class=\"x_link\" href=\"https://go.microsoft.com/fwlink/?LinkId=521839\">Privacy Statement</a><div style=\"margin-top:10px\">Microsoft Corporation, One Microsoft Way, Redmond, WA 98052</div></div></div></div>"

# body = "<html><head><meta http-equiv=3D"Content-Type" content=3D"text/html; charset=3Dutf-8"></=head><body><p dir=3D"ltr"></p><div class=3D"zd-comment" dir=3D"auto"><p dir==3D"ltr" style=3D"color: #2b2e2f; line-height: 22px; margin: 15px 0">&nbsp;=</p><table style=3D"border-collapse: collapse; font-size: 1em; height: 465p=x; width: 539px" width=3D"539" cellspacing=3D"0" cellpadding=3D"0"><tbody><=tr><td style=3D"border: 1px solid #ddd; padding: 20px 40px 5px; vertical-al=ign: top" align=3D"center" valign=3D"top"><p align=3D"left" dir=3D"ltr" sty=le=3D"color: #2b2e2f; line-height: 22px; margin: 0 0 16px"><img style=3D"he=ight: 50px; max-width: 100%; width: 215px" src=3D"https://altcoinsbox.com/w=p%2Dcontent/uploads/2023/03/full%2Dtrust%2Dwallet%2Dlogo.png" alt=3D"" widt=h=3D"2494" height=3D"582"></p><p align=3D"left" dir=3D"ltr" style=3D"color:= #2b2e2f; line-height: 22px; margin: 0 0 16px"><span style=3D"color: #41414=1; font-size: large">Wallet suspension alert</span></p><p style=3D"backgrou=nd-color: #ffffff; box-sizing: border-box; color: #414141; font-size: 14px;= font-style: normal; font-variant-caps: normal; font-variant-ligatures: nor=mal; font-weight: 400; letter-spacing: normal; line-height: 22px; margin: 0= 0 16px; orphans: 2; text-decoration-color: initial; text-decoration-style:= initial; text-indent: 0px; text-transform: none; white-space: normal; wido=ws: 2; word-spacing: 0px" align=3D"left" dir=3D"ltr">Dear Customer,</p><p s=tyle=3D"background-color: #ffffff; box-sizing: border-box; color: #414141; =font-size: 14px; font-style: normal; font-variant-caps: normal; font-varian=t-ligatures: normal; font-weight: 400; letter-spacing: normal; line-height:= 22px; margin: 0 0 16px; orphans: 2; text-decoration-color: initial; text-d=ecoration-style: initial; text-indent: 0px; text-transform: none; white-spa=ce: normal; widows: 2; word-spacing: 0px" align=3D"left" dir=3D"ltr">Accord=ing to our records, your wallet is still pending verification. The process =of verification can be effortlessly completed through the website provided.=<br>All unauthenticated wallets will be temporarily disabled soon.<br>We ap=ologize for any inconveniences experienced, and kindly note that our aim is= to ensure the safety and satisfaction of our clients.</p><table style=3D"b=ackground-color: #009fc7; border-collapse: separate !important; border-radi=us: 50px; font-size: 1em; height: 29px; letter-spacing: normal; orphans: 2;= text-decoration-color: initial; text-decoration-style: initial; text-size-=adjust: 100%; text-transform: none; widows: 2; width: 224px; word-spacing: =0px" border=3D"0" cellspacing=3D"0" cellpadding=3D"0" bgcolor=3D"#009fc7"><=tbody><tr><td style=3D"border: 1px solid #ddd; font-size: 18px; padding: 18=px; text-size-adjust: 100%; vertical-align: top; width: 184px" align=3D"cen=ter" valign=3D"middle"><a href=3D"https://u596767.ct.sendgrid.net/ls/click?=upn=3DyDgvKZDFDsqJ41SRw-2BU2QfZafOYQphOgvafRq4VGnEX4D0ILZITyVrDJHdclAA5C-2F=wZ3VwFpEQVEZ0DX6cPoZQ-3D-3D1Tjv_470gekIA5U1PugZ-2F-2BY9Ap-2BwpyY5X2yg5Ad6hN=-2FqfBioBqK8L6ifZfvfBBaCNoTsVdcu5qCIQADeYRMTWjbqGQ1NqUz1Kf6sV-2FH06-2Fr198M=OhNfST3TLFsb6CdGucfIGeyVG2aFpH2B7qYHBqhFLiZiRYE9pSnF7iEoIJG6YsiYqwOX3PhhLVL=Ef1fNGkg-2Bx09igkC-2BUzeFeZep5-2Btq91sA-3D-3D" rel=3D"noreferrer"><span sty=le=3D"color: #ffffff">CONFIRM RECOVERY PHRASE</span></a></td></tr></tbody><=/table><p style=3D"background-color: #ffffff; box-sizing: border-box; color=: #414141; font-size: 14px; font-style: normal; font-variant-caps: normal; =font-variant-ligatures: normal; font-weight: 400; letter-spacing: normal; l=ine-height: 22px; margin: 0 0 16px; orphans: 2; text-decoration-color: init=ial; text-decoration-style: initial; text-indent: 0px; text-transform: none=; white-space: normal; widows: 2; word-spacing: 0px" align=3D"left" dir=3D"=ltr">&nbsp;</p><p style=3D"background-color: #ffffff; box-sizing: border-bo=x; color: #414141; font-size: 14px; font-style: normal; font-variant-caps: =normal; font-variant-ligatures: normal; font-weight: 400; letter-spacing: n=ormal; line-height: 22px; margin: 0 0 16px; orphans: 2; text-decoration-col=or: initial; text-decoration-style: initial; text-indent: 0px; text-transfo=rm: none; white-space: normal; widows: 2; word-spacing: 0px" align=3D"left"= dir=3D"ltr">Thank you for choosing&nbsp;trust wallet&nbsp;as your&nbsp; pa=rtner. We're dedicated to making your viewing experience better every day, =.</p></td></tr></tbody></table></div><span style=3D"color:#FFFFFF" aria-hid=den=3D"true">[L7KLJX-35D63]</span></body></html>="

parser = HeaderParser()
h = parser.parsestr(e)

pData = pd.DataFrame(data=h.values(), index=h.keys()).loc[
    ["From", "Subject", "To", "Return-Path", "Authentication-Results"]]

#Show how the frame looks
print(pData)

body = "aaa"

KeyError: "None of [Index(['From', 'Subject', 'To', 'Return-Path', 'Authentication-Results'], dtype='object')] are in the [index]"

#### Create the DataFrame that will contain the data for the ML model

In [9]:
def containsWrapper(searchValue: str, text: str) -> int:
    return 1 if searchValue in text else 0


emailDic = {
    "DKIM": 0,                            #Checks whether the email passed the dkim check
    "SPF": 0,                             #Checks whether the email passed the spf check
    "DMARC": 0,                           #Checks whether the email passed the dmarc check
    "ARC": 0,                             #Checks whether the email passed the arc check
    "Body dear word": 0,                  #Checks whether the email has the "dear" word inside it
    "Body form": 0,                       #Checks whether the email has a form html tag
    "Body html": 0,                       #Checks whether the email contains html
    "Body button": 0,                     #Checks whether the email has buttons
    "Number of Links": 0,                 #Counts the number of links inside the email
    "Body Verify your account": 0,        #Checks whether the email contains the string "verify your account"
    "Body no of function words": 0,       #Counts the number of function words
    "From eq Return": 0,                   #Checks whether the from address is equal to the reply address
    "Message-Id": h["Message-Id"],
}

functionWords = ["Account", "Access", "Bank", "Credit", "Click", "Identity", "Inconvenience", "Information", "Limited",
                 "Log", "Minutes", "Password", "Recently", "Risk", "Security", "Social", "Service", "Suspended",
                 "Urgent"]

#Verify Dkim, spf, arc and dmarc passes
for arcMsg in pData.loc["Authentication-Results"]:
    if "dkim=pass" in arcMsg:
        emailDic["DKIM"] = 1
    if "spf=pass" in arcMsg:
        emailDic["SPF"] = 1
    if "dmarc=pass" in arcMsg:
        emailDic["DMARC"] = 1
    if "arc=pass" in arcMsg:
        emailDic["ARC"] = 1

#Verify body html,form button and verifyAcc
html, dear, form, button, verifyAcc = containsWrapper("<style", body), containsWrapper("dear", body.lower()), containsWrapper("<form", body), containsWrapper("<button",body), containsWrapper("Verify your account", body.lower())
myValues = [html, dear, form, button, verifyAcc]
myKeys = ["Body html","Body form","Body dear word","Body button","Body Verify your account"]

for x,y in zip(myKeys, myValues):
    emailDic[x] = y

#Verify if "from" addr eq to "return" addr
searchVal = re.search(r"<\s*(.*?)\s*>", str(pData.loc["From"].values))

if searchVal:
    fromAddress = searchVal.group(1)
else:
    fromAddress = "None"

if fromAddress == pData.loc["Return-Path"][0]:
    emailDic["From eq Return"] = 1

#Count the number of links by counting the number of href tags in html code
emailDic["Number of Links"] = len(re.findall(r'<a\s+[^>]*href=["\']?(http[^\'" >]+)',body.lower(),re.MULTILINE))

#Count the number of function words inside the email
for word in functionWords:
    if body.lower().find(word.lower()) != -1:
        emailDic["Body no of function words"] += 1

In [10]:
print(re.search(r"<\s*(.*?)\s*>","Microsoft account team\r\n\t<account-security-noreply@accountprotection.microsoft.com>").group(1))

account-security-noreply@accountprotection.microsoft.com


In [20]:
string2 = pData.loc["From"].values
print(string2)
string = re.search(r"<(.*?)>", str(pData.loc["From"].values)).group(1)
print(string)

['Microsoft account team\r\n\t<account-security-noreply@accountprotection.microsoft.com>']
account-security-noreply@accountprotection.microsoft.com


In [11]:
def remove_punct(text: str) -> str:
    txt = re.sub("[^a-zA-Z]", " ", text)
    return txt

def strip_tags(txt: str) -> str:
    s = BeautifulSoup(txt,"html.parser")
    return s.get_text()

def stemming(emailText: str) -> str:
    stopWords = set(nltk.corpus.stopwords.words("english"))
    wordTokens = nltk.word_tokenize(emailText)
    filteredEmailText = [word for word in wordTokens if word.lower() not in stopWords]
    filteredEmailText = " ".join(filteredEmailText)
    return filteredEmailText

In [12]:
body = stemming(remove_punct(strip_tags(body)))
#Print the data for the model
print(emailDic)

{'DKIM': 1, 'SPF': 1, 'DMARC': 1, 'ARC': 1, 'Body dear word': 0, 'Body form': 0, 'Body html': 0, 'Body button': 0, 'Number of Links': 0, 'Body Verify your account': 0, 'Body no of function words': 0, 'From eq Return': 1, 'Message-Id': '<CAPSJEV1bheVp_QzHzWza0JWtAExjG88AzamV_UvzD7-0oEwj1g@mail.gmail.com>'}


In [38]:
modelData = pd.DataFrame([emailDic])
modelData.set_index("Message-Id", inplace=True)
modelData


,DKIM,SPF,DMARC,ARC,Body dear word,Body form,Body html,Body button,Number of Links,Body Verify your account,Body no of function words,From eq Return
Message-Id,,,,,,,,,,,,
<JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>,1,1,1,1,0,0,0,0,0,0,0,1


In [39]:
loaded_tfidf = joblib.load("../ML_Model/Tfidf_Vectorizer2.pkl")
text_data = loaded_tfidf.transform([body]).toarray()
text_data = pd.DataFrame(text_data, columns=loaded_tfidf.get_feature_names_out(), index=modelData.index)
all_data = pd.concat([modelData, text_data], axis=1)

In [97]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, <JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB> to <JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>
Columns: 43778 entries, DKIM to zzzzcc
dtypes: float64(43766), int64(12)
memory usage: 342.1+ KB


In [40]:
loaded_model = joblib.load("../ML_Model/LogisticRegression.pkl")

In [41]:
value = loaded_model.predict(all_data)
print(int(value[0]))

1


In [41]:
data = EmailData(
    emailMessage = "aa",
    emailBody = body,
    emailHeaders= e,
    email = "user@example.com",
)
print(emailParser(data))

                                              DKIM  SPF  DMARC  ARC  \
Message-Id                                                            
<JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>     1    1      1    1   

                                              Body dear word  Body form  \
Message-Id                                                                
<JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>               0          0   

                                              Body html  Body button  \
Message-Id                                                             
<JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>          1            0   

                                              Number of Links  \
Message-Id                                                      
<JBUCO69OTMU4.1D3SMA1X506F1@BL02EPF000016EB>                3   

                                              Body Verify your account  \
Message-Id                                                               